In [106]:
import pandas as pd
import numpy as np
import glob
import re

We first merge individual CSVs to create a tidy dataset. Our units of interest (comprising one row) consist of unique combinations of state, gender, and year. Since the goal of this project is to look at recidivism, we are interested only in convicted individuals in prison. We have caste, education, and other demographic information for only a subset of the 

In [154]:
fileList = glob.glob("./Data/*.csv")
dfNames = [re.search("[(\\\)](.*)\.",file).group(1) for file in fileList]

for idx in range(len(dfNames)):
    df = pd.read_csv(fileList[idx])
    name = dfNames[idx]
    vars()[name] = df
    print(vars()['name'])


Age_group
Area
Caste
Death_sentence
Domicile
Education
Education_facilities
Inmates_death
Inmates_escapee
IPC_crime_inmates_convicted
IPC_crime_inmates_under_trial
Jail_wise_population_of_prison_inmates
pop
Prison_details_2015
Recidivism
Rehabilitation
Religion
Sentence_period
tidy_df
Tranquillity
Vocational_training
Wages


## IPC Crimes (Convicted Prisoners)
- aggregated into Theft, Business, Sexual, and Violent crimes
- column names prefixed by IPC

In [ ]:
IPC_crime_inmates_convicted.groupby(["crime_head"]).sum(). \
sort_values(by=['Grand Total'])

In [ ]:
ViolentCrimes=["Murder","Attempt To Commit Murder","C.H. Not Amounting To Murder","Kidnapping And Abduction","Dacoity","Arson"]
SexualCrimes=["Rape","Dowry Deaths","Cruelty By Husband Or Relative Of Husband","Molestation","Eve-Teasing"]
BusinessCrimes=["Cheating","Counter Feiting","Criminal Breach Of Trust"]
TheftCrimes=["Thefts","Robbery","Burglary", "Prep. And Assembly For Dacoity", "Extortion"]

IPC_crime_inmates_convicted_fin = IPC_crime_inmates_convicted.assign(CrimeType= #aggregating crimes into crime types
                                   np.select([IPC_crime_inmates_convicted["crime_head"].isin(ViolentCrimes),
                                             IPC_crime_inmates_convicted["crime_head"].isin(SexualCrimes),
                                             IPC_crime_inmates_convicted["crime_head"].isin(BusinessCrimes),
                                             IPC_crime_inmates_convicted["crime_head"].isin(TheftCrimes)],
                                            ["Violent","Sexual","Business","Theft"],
                                            default="Unknown")). \
filter(["state_name","year","Total 16-18 years", "Total 18-30 years","Total 30-50 years","Total Above 50 years","CrimeType"]). \
groupby(["state_name","year","CrimeType"]).sum(). \
pivot_table(index=["state_name","year"],
           columns=["CrimeType"]). \
assign(All=lambda df: df.sum(axis=1))


IPC_crime_inmates_convicted_fin=IPC_crime_inmates_convicted_fin.div(IPC_crime_inmates_convicted_fin['All'],axis=0). \
drop(['All'],axis=1)

In [163]:
IPC_crime_inmates_convicted_fin

IPC-prop-Business-Total 16-18 years  \
state_name    year                                        
A & N Islands 2001                                  0.0   
              2002                                  0.0   
              2003                                  0.0   
              2004                                  0.0   
              2005                                  0.0   
...                                                 ...   
West Bengal   2009                                  0.0   
              2010                                  0.0   
              2011                                  0.0   
              2012                                  0.0   
              2013                                  0.0   

                    IPC-prop-Sexual-Total 16-18 years  \
state_name    year                                      
A & N Islands 2001                                0.0   
              2002                                0.0   
              2003                                0.0   
              2004                                0.0   
              2005                                0.0   
...                                               ...   
West Bengal   2009                                0.0   
              2010                                0.0   
              2011                                0.0   
              2012                                0.0   
              2013                                0.0   

                    IPC-prop-Theft-Total 16-18 years  \
state_name    year                                     
A & N Islands 2001                               0.0   
              2002                               0.0   
              2003                               0.0   
              2004                               0.0   
              2005                               0.0   
...                                              ...   
West Bengal   2009                               0.0   
              2010                               0.0   
              2011                               0.0   
              2012                               0.0   
              2013                               0.0   

                    IPC-prop-Unknown-Total 16-18 years  \
state_name    year                                       
A & N Islands 2001                                 0.0   
              2002                                 0.0   
              2003                                 0.0   
              2004                                 0.0   
              2005                                 0.0   
...                                                ...   
West Bengal   2009                                 0.0   
              2010                                 0.0   
              2011                                 0.0   
              2012                                 0.0   
              2013                                 0.0   

                    IPC-prop-Violent-Total 16-18 years  \
state_name    year                                       
A & N Islands 2001                                 0.0   
              2002                                 0.0   
              2003                                 0.0   
              2004                                 0.0   
              2005                                 0.0   
...                                                ...   
West Bengal   2009                                 0.0   
              2010                                 0.0   
              2011                                 0.0   
              2012                                 0.0   
              2013                                 0.0   

                    IPC-prop-Business-Total 18-30 years  \
state_name    year                                        
A & N Islands 2001                             0.000000   
              2002                             0.000000   
              2003                 

In [ ]:
#IPC_crime_inmates_convicted_fin.columns = [' '.join(col).strip() for col in IPC_crime_inmates_convicted_fin.columns.values]

valColNames=["IPC-prop-{}-{}".format(b_, a_) for a_, b_ in zip(IPC_crime_inmates_convicted_fin.columns.get_level_values(0),
         IPC_crime_inmates_convicted_fin.columns.get_level_values(1))]

IPC_crime_inmates_convicted_fin.columns=valColNames
IPC_crime_inmates_convicted_fin

# All-Surveyed-Prisoner Demographics

## Jail-Wise Population
- breakdown of surveyed prisoners among different jail types
- column names prefixed by JWP

In [ ]:
Jail_wise_population_of_prison_inmates_fin = Jail_wise_population_of_prison_inmates[["state_name","year","Jail_Type","Total_Convicts"]]. \
pivot_table(index=["state_name","year"], 
           columns="Jail_Type", values="Total_Convicts")

Jail_wise_population_of_prison_inmates_fin=Jail_wise_population_of_prison_inmates_fin. \
div(Jail_wise_population_of_prison_inmates_fin['Total'], axis=0). \
drop(['Total'],axis=1)

Jail_wise_population_of_prison_inmates_fin.columns= ["JWP-prop-"+s for s in Jail_wise_population_of_prison_inmates_fin.columns]
Jail_wise_population_of_prison_inmates_fin

## Religion
- column names prefixed by REL

In [ ]:
Religion_fin = Religion.query("is_state==1"). \
drop(["is_state","under_trial","detenues","others"],axis=1). \
pivot_table(index=["state_name","year"],
           columns=["gender","religion"],
           values="convicts"). \
assign(All=lambda df: df.sum(axis=1))

Religion_fin=Religion_fin.div(Religion_fin['All'], axis=0). \
drop(['All'],axis=1)

valColNames=["REL-prop-{}-{}".format(b_, a_) for a_, b_ in zip(Religion_fin.columns.get_level_values(0),
         Religion_fin.columns.get_level_values(1))]

Religion_fin.columns=valColNames

Religion_fin

In [ ]:
Religion.query("is_state==1"). \
drop(["is_state","under_trial","detenues","others"],axis=1). \
groupby(["state_name","year","gender"]).sum()

## Education
- column names prefixed by EDUC

In [ ]:
Education_fin = Education.query("is_state==1"). \
drop(["is_state","under_trial","detenues","others"],axis=1). \
pivot_table(index=["state_name","year"],
           columns=["gender","education"],
           values="convicts"). \
assign(All=lambda df: df.sum(axis=1))

Education_fin=Education_fin.div(Education_fin['All'], axis=0). \
drop(['All'],axis=1)

valColNames=["EDUC-prop-{}-{}".format(b_, a_) for a_, b_ in zip(Education_fin.columns.get_level_values(0),
         Education_fin.columns.get_level_values(1))]

Education_fin.columns=valColNames

Education_fin

In [ ]:
Education.query("is_state==1"). \
drop(["is_state","under_trial","detenues","others"],axis=1). \
groupby(["state_name","year","gender"]).sum()

## Sentence Period

- Missing values in the wide-pivoted data are equivalent to zero counts. 
- Total counts of female and male prisoners by age group are usually larger than in the Age_group dataframe
- Column names prefixed by SP


In [ ]:
Sentence_period.loc[Sentence_period['sentence_period']=='10  Less than 3 months','sentence_period']='Less than 3 months'
Sentence_period.loc[Sentence_period['sentence_period']=='3 less than 6 months','sentence_period']='3 - 6 months'

In [ ]:
Sentence_period_fin = Sentence_period.query('is_state==1'). \
drop(["is_state"],axis=1). \
pivot_table(index=["state_name","year"], 
                   columns = ["gender","sentence_period"]). \
fillna(0). \
assign(All=lambda df: df.sum(axis=1))

Sentence_period_fin=Sentence_period_fin.div(Sentence_period_fin['All'], axis=0). \
drop(['All'],axis=1)

valColNames=["SP-prop-{}-{}-{}".format(c_,b_, a_) for a_, b_, c_ in zip(Sentence_period_fin.columns.get_level_values(0),
         Sentence_period_fin.columns.get_level_values(1),Sentence_period_fin.columns.get_level_values(2))]

Sentence_period_fin.columns=valColNames



In [ ]:
life_sentence = ["Capital Punishment","Life Imprisonment"]
long_sentence = ['10-13 Plus years',
       '7-9 Plus years', '5-6 Plus years']
med_sentence = ['2-4 Plus years',
       '1 Less than 2 years']
light_sentence = ['6 months less than 1 Yr.', '3 - 6 months',
       'Less than 3 months']

Sentence_period_fin=Sentence_period.query('is_state==1'). \
drop(["is_state"],axis=1). \
assign(sentence=lambda df: np.select([df["sentence_period"].isin(life_sentence),
                          df["sentence_period"].isin(long_sentence),
                          df["sentence_period"].isin(med_sentence),
                          df["sentence_period"].isin(light_sentence)],
                         ['life','long','med','light'],
                         default=None)). \
drop(["sentence_period"],axis=1). \
groupby(['state_name','year','gender','sentence']).sum(). \
pivot_table(index = ['state_name','year'], columns=['gender','sentence']). \
assign(All=lambda df: df.sum(axis=1))

Sentence_period_fin=Sentence_period_fin.div(Sentence_period_fin['All'], axis=0). \
drop(['All'],axis=1)

valColNames=["SP-prop-{}-{}-{}".format(c_,b_, a_) for a_, b_, c_ in zip(Sentence_period_fin.columns.get_level_values(0),
         Sentence_period_fin.columns.get_level_values(1),Sentence_period_fin.columns.get_level_values(2))]

Sentence_period_fin.columns=valColNames

Sentence_period_fin=Sentence_period_fin. \
filter(Sentence_period_fin.loc[:,Sentence_period_fin.columns.str.contains("16_18")==False])



In [ ]:
Sentence_period_fin.filter(regex=r"^.*Male-age_50_above$")

In [ ]:
Sentence_period.query('is_state==1').drop(['is_state'],axis=1).groupby(["state_name","year","gender"]).sum(). \
assign(total= lambda df: df.iloc[:,:].sum(axis=1))

## Age_group
- Same information (but somewhat mismatched and lower prisoner counts) as sentence period
- Do not use this dataframe

In [ ]:
Age_group.query('type=="Convicts" and is_state==1 and category!="Foreigners"'). \
drop(['is_state','category','type'],axis=1). \
assign(total= lambda df: df.iloc[:,3:].sum(axis=1))

## Caste

- SC/ST refers to class traditionally known as untouchables/Dalits
- OBC is an umbrella class of other economically/socially disadvantaged citizens
- Others refers to relatively well-off citizens
- column names prefixed by CTE

In [ ]:
Caste_fin = Caste.iloc[:,:6].query('is_state==1'). \
pivot_table(index=["state_name","year"], 
                   columns = ["gender","caste"],
                   values="convicts"). \
assign(All=lambda df: df.sum(axis=1))

Caste_fin=Caste_fin.div(Caste_fin['All'], axis=0). \
drop(['All'],axis=1)

valColNames=["CST-prop-{}-{}".format(b_, a_) for a_, b_ in zip(Caste_fin.columns.get_level_values(0),
         Caste_fin.columns.get_level_values(1))]

Caste_fin.columns=valColNames

Caste_fin

In [ ]:
Caste.query('is_state==1').drop(['is_state','under_trial','detenues','others'],axis=1). \
groupby(["state_name","year","gender"]).sum() 

## Population/Density
- density is proxy for rural or urban regions

In [160]:
pop_area=pd.merge(pop, Area, on="state_name"). \
assign(dens=lambda df: df["pop_2011"]/df["area"])

# Ambiguous Count (from surveyed or population?)

In [ ]:
Education_facilities

## Tranquility

- aggregated over prisoner and personnel injuries/deaths as well as types of unrest
- about 3/4 of regions have 0 recorded incidents of violence 

In [ ]:
Tranquility_fin = Tranquillity.assign(injured=lambda df: df["inmate_injured"]+df["jail_personnel_injured"],
                   killed=lambda df: df["inmate_killed"]+df["jail_personnel_killed"]). \
drop(["inmate_injured","jail_personnel_injured","inmate_killed","jail_personnel_killed"],axis=1). \
groupby(["state_name","year"]).sum()

## Vocational Training
- Hundreds of unique training jobs; need to compress information to keep model identifiable
- Some prisoners are in multiple training problems: in Punjab, for example, 
- For now, I use total number of inmates getting vocational training as a variable, not accounting for double counting

In [ ]:
Vocational_training_fin = Vocational_training.groupby(["state_name","year"]).sum()
#.pivot_table(index=["state_name","year"],
#                                columns="vocational_trainings_program",
#                                values="inmates_trained")

In [ ]:
len(Vocational_training.vocational_trainings_program.unique())

## Rehabilitation
- This dataframe is already in a nice wide form.

In [ ]:
Rehabilitation

## Recidivism

I mutate the recidivism ratio (number of habitual offenders over number of admitted convicts) for each state and year.

In [ ]:
Recidivism_fin = Recidivism.assign(recidiv_ratio= \
                  lambda df: df["habitual_offenders"]/df["convicts_admitted"])
Recidivism_fin

In [161]:
tidy_Indian_Prison_df = pd.merge(Recidivism_fin, Caste_fin, on=["state_name","year"]). \
merge(Sentence_period_fin, on=["state_name","year"]). \
merge(Education_fin, on=["state_name","year"]). \
merge(Religion_fin, on=["state_name","year"]). \
merge(IPC_crime_inmates_convicted_fin,on=["state_name","year"]). \
merge(Jail_wise_population_of_prison_inmates_fin, on=["state_name","year"]). \
merge(Education_facilities, on=["state_name","year"]). \
merge(Vocational_training_fin, on=["state_name","year"]). \
merge(Rehabilitation, on=["state_name","year"]). \
merge(Tranquility_fin, on=["state_name","year"]). \
merge(pop_area, on=["state_name"])
tidy_Indian_Prison_df.to_csv("Data/tidy_df.csv", index=False)

In [162]:
tidy_Indian_Prison_df

,state_name,year,convicts_admitted,habitual_offenders,recidiv_ratio,CST-prop-OBC-Female,CST-prop-Others-Female,CST-prop-SC-Female,CST-prop-ST-Female,CST-prop-OBC-Male,...,financial_assistance_provided,rehabilitated,legal_aid_provided,incidence,injured,killed,pop_2011,area,region,dens
0,Andhra Pradesh,2001,17345,1063,0.061286,0.011846,0.002843,0.007818,0.001658,0.361526,...,14,75,2594,0,0,0,49386799,160205,Southern,308.27252
1,Andhra Pradesh,2002,13322,318,0.023870,0.013342,0.005952,0.006979,0.004721,0.376232,...,37,108,1876,0,0,0,49386799,160205,Southern,308.27252
2,Andhra Pradesh,2003,15682,536,0.034179,0.020400,0.006023,0.004857,0.002137,0.442782,...,0,0,2126,0,0,0,49386799,160205,Southern,308.27252
3,Andhra Pradesh,2004,14397,399,0.027714,0.024344,0.006506,0.007345,0.003148,0.450367,...,0,0,909,0,0,0,49386799,160205,Southern,308.27252
4,Andhra Pradesh,2005,12389,977,0.078860,0.023947,0.003792,0.008182,0.003592,0.439234,...,0,204,2342,0,0,0,49386799,160205,Southern,308.27252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,West Bengal,2009,10890,2100,0.192837,0.004899,0.044095,0.011826,0.007603,0.067748,...,161,5,2920,1,6,0,91347736,88752,Eastern,1029.24707
314,West Bengal,2010,12150,1940,0.159671,0.008827,0.041537,0.011596,0.003981,0.105054,...,170,12,3020,1,2,0,91347736,88752,Eastern,1029.24707
315,West Bengal,2011,10259,1470,0.143289,0.006007,0.042403,0.010424,0.005124,0.056360,...,37,4,4057,3,36,0,91347736,88752,Eastern,1029.24707
316,West Bengal,2012,8650,1207,0.139538,0.002431,0.051531,0.007940,0.003565,0.074380,...,25,23,2583,7,14,0,91347736,88752,Eastern,1029.24707
